In [80]:
import re
import math
import itertools

In [95]:
data = open('data/day11.txt', 'r').read()

In [3]:
example = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""

In [11]:
def print_grid(grid):
    """
    Print the grid
    """
    for row in range(len(grid)):
        print(''.join(grid[row]))

In [74]:
def cosmic_expansion(data):
    """
    Any rows or columns with no galaxies ('#') need to be twice as big
    """
    grid = [list(line) for line in data.split('\n')]
    # Expand rows in place
    num_rows_expanded = 0
    row_index = 0
    while (row_index + num_rows_expanded) < len(grid):
        if len(re.findall('#', ''.join(grid[row_index + num_rows_expanded]))) == 0:
            grid = grid[:row_index + num_rows_expanded] + [grid[row_index + num_rows_expanded]] + grid[row_index + num_rows_expanded:]
            num_rows_expanded += 1
        row_index += 1

    # Expand columns in place
    num_cols_expanded = 0
    col_index = 0
    while (col_index + num_cols_expanded) < len(grid[0]):
        row_values = [row[col_index + num_cols_expanded] for row in grid]
        if len(re.findall('#', ''.join(row_values))) == 0:
            for row_index, row in enumerate(grid):
                grid[row_index] = row[:col_index + num_cols_expanded] + [row[col_index + num_cols_expanded]] + row[col_index + num_cols_expanded:]
            num_cols_expanded += 1
        col_index += 1
                
    return grid

In [89]:
def find_galaxy_pairs(grid):
    """
    Find the unique pairs of coordinates of all the galaxies ('#') in the grid
    """
    # Find all galaxies ('#')
    galaxies = []
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            if grid[row][col] == '#':
                galaxies.append((row, col))
    
    # Find unique combinations of galaxy pairs
    galaxy_pairs = list(itertools.combinations(galaxies, 2))
    
    return galaxy_pairs

In [90]:
def find_best_step(grid, source, dest):
    """
    Find the best step that decreases the distance between the source and dest
    """
    movements = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1)}
    
    distance = math.inf
    best_step = None
    source_row, source_col = source
    dest_row, dest_col = dest
    for movement, (row_movement, col_movement) in movements.items():
        step = (step_row, step_col) = (source_row + row_movement, source_col + col_movement)
        distance_to_dest = ((dest_row - step_row) ** 2 + (dest_col - step_col) **2) ** 0.5
        if distance_to_dest < distance:
            distance = distance_to_dest
            best_step = step
    
    return best_step

In [91]:
def find_shortest_path(grid, source, dest):
    """
    Find the shortest path between the source and dest
    """
    source_row, source_col = source
    dest_row, dest_col = dest
    
    total_steps = 0
    curr_step = source
    while curr_step != dest:
        curr_step = find_best_step(grid, curr_step, dest)
        total_steps += 1
    
    return total_steps

In [96]:
def part1(data):
    # Perform cosmic expansion
    grid = cosmic_expansion(data)
    # Find galaxy pairs
    galaxy_pairs = find_galaxy_pairs(grid)
    # Find the shortest path for each pair
    path_lengths = {(source, dest): find_shortest_path(grid, source, dest) for (source, dest) in galaxy_pairs}
    
    return path_lengths
sum(part1(data).values())

9918828

In [101]:
def track_cosmic_expansion(data):
    """
    Find all rows or columns with no galaxies ('#')
    """
    grid = [list(line) for line in data.split('\n')]
    row_expansions = []
    # Track rows that need to be expanded
    for row_index, row in enumerate(grid):
        if len(re.findall('#', ''.join(row))) == 0:
            expanded_row = [(row_index, col_index) for col_index in range(len(grid[row_index]))]
            row_expansions += expanded_row

    # Track columns that need to be expanded
    col_expansions = []
    for col_index in range(len(grid[0])):
        row_values = [row[col_index] for row in grid]
        if len(re.findall('#', ''.join(row_values))) == 0:
            expanded_col = [(row_index, col_index) for row_index in range(len(grid))]
            col_expansions += expanded_col
                
    return grid, row_expansions, col_expansions

In [102]:
def find_best_movement_step(grid, source, dest):
    """
    Find the best movement direction and step that decreases the distance between the source and dest
    """
    movements = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1)}
    
    distance = math.inf
    best_step = None
    best_movement_direction = None
    source_row, source_col = source
    dest_row, dest_col = dest
    for movement, (row_movement, col_movement) in movements.items():
        step = (step_row, step_col) = (source_row + row_movement, source_col + col_movement)
        distance_to_dest = ((dest_row - step_row) ** 2 + (dest_col - step_col) **2) ** 0.5
        if distance_to_dest < distance:
            distance = distance_to_dest
            best_step = step
            best_movement_direction = movement
    
    return best_movement_direction, best_step

In [133]:
def find_shortest_path_with_multiplier(grid, row_expansions, col_expansions, source, dest, multiplier):
    """
    Find the shortest path between the source and dest, accounting for the expansion multiplier when calculating steps
    """
    expansion_checks = {'Up': row_expansions, 'Down': row_expansions, 'Left': col_expansions, 'Right': col_expansions}

    source_row, source_col = source
    dest_row, dest_col = dest
    
    total_steps = 0
    curr_step = source
    while curr_step != dest:
        movement_direction, curr_step = find_best_movement_step(grid, curr_step, dest)
        expansion_check = expansion_checks[movement_direction]
        if curr_step in expansion_check:
            total_steps += multiplier
        else:
            total_steps += 1
    
    return total_steps

In [137]:
def part2(data, multiplier):
    """
    Takes about 2 minutes 15 for 1M
    """
    # Track cosmic expansion
    grid, row_expansions, col_expansions = track_cosmic_expansion(data)
    # Find galaxy pairs
    galaxy_pairs = find_galaxy_pairs(grid)
    # Find the shortest path for each pair
    total_path_length = sum([find_shortest_path_with_multiplier(grid, row_expansions, col_expansions, source, dest, multiplier) 
                             for (source, dest) in galaxy_pairs])

    return total_path_length

part2(data, 1000000)

692506533832